In [1]:
import matplotlib as mpl
import matplotlib.cm
import matplotlib.colorbar
import matplotlib.pyplot as plt
import matplotlib.patches as pc
from matplotlib.path import Path

import numpy as np
import re
import pandas as pd


mpl.rcParams['pdf.fonttype'] = 42
mpl.rc('text', usetex=True)
mpl.rc('text.latex', preamble=r'\usepackage{amsmath}')
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'

%matplotlib qt

In [2]:
sq_property_diverging_flow = pd.read_excel('diverging_flow_sq_property.xlsx', index_col=0)
tq_property_diverging_flow = pd.read_excel('diverging_flow_algorithm_graph_error.xlsx', index_col=0, sheet_name=None)
sq_property_vortex = pd.read_excel('vortex_sq_property.xlsx', index_col=0)
tq_property_vortex = pd.read_excel('vortex_algorithm_graph_error.xlsx', index_col=0, sheet_name=None)

In [3]:
SIZE = [21, 21]
def adjacentQubit(name):
    pattern = re.compile(r'(q|c)(\d+)_(\d+)')
    device, idx1, idx2 = pattern.findall(name)[0]
    idx1, idx2 = int(idx1), int(idx2)
    if device == 'c':
        if idx1 % 2 == 1:
            return [f'q{idx1}_{idx2-1}', f'q{idx1}_{idx2+1}']
        else:
            return [f'q{idx1-1}_{idx2}', f'q{idx1+1}_{idx2}']
    else:
        assert idx1 % 2 == 1
        assert idx2 % 2 == 1
        qubit_names = [None] * 4
        if idx1 < SIZE[0]:
            qubit_names[0] = f'q{idx1+2}_{idx2}'
        if idx1 > 1:
            qubit_names[1] = f'q{idx1-2}_{idx2}'
        if idx2 > 1:
            qubit_names[2] = f'q{idx1}_{idx2-2}'
        if idx2 < SIZE[1]:
            qubit_names[3] = f'q{idx1}_{idx2+2}'
        return qubit_names


In [4]:
def CZ_error(cycle_error, sq_error1, sq_error2):
    return 1-(1-cycle_error)/(1-sq_error1)/(1-sq_error2)
cz_error_diverging_flow = tq_property_diverging_flow['CZ'].copy().T
for cname in cz_error_diverging_flow:
    qnames = adjacentQubit(cname)
    cz_error_diverging_flow[cname] = CZ_error(cz_error_diverging_flow[cname], sq_property_diverging_flow['Single-qubit gate error'][qnames[0]], sq_property_diverging_flow['Single-qubit gate error'][qnames[1]])
    
cz_error_vortex = tq_property_vortex['CZ'].copy().T
for cname in cz_error_vortex:
    qnames = adjacentQubit(cname)
    cz_error_vortex[cname] = CZ_error(cz_error_vortex[cname], sq_property_vortex['Single-qubit gate error'][qnames[0]], sq_property_vortex['Single-qubit gate error'][qnames[1]])


In [5]:
sq_property = sq_property_diverging_flow

In [6]:
figsize = np.array([180, 50]) / 25.4
fontsize=8
fig = plt.figure(figsize=[figsize[0], figsize[1]])
ax_xs = [0.08, 0.28, 0.48, 0.68]
ax_ys = [0.19]*4
ax_width = [0.17, 0.17, 0.17, 0.3]
ax_height = [0.71]*4
# f10 ecdf
ax_idx = 0
ax = fig.add_axes([ax_xs[ax_idx], ax_ys[ax_idx], ax_width[ax_idx], ax_height[ax_idx]])
f10_array = np.sort(np.array(sq_property['Qubit f10 (GHz)']))
y = np.arange(len(f10_array))/len(f10_array)
ax.step(np.hstack([f10_array, f10_array[-1]]), np.hstack([y, 1]), color='C0')
ax.plot([np.median(f10_array)]*100, np.linspace(0, 1, 100), color='C0', ls='--')
ax.set_xlim(3.7, 4.5)
ax.set_ylim(0, 1)
plt.xticks([3.8, 4, 4.2, 4.4], fontsize=fontsize)
plt.yticks([0, 0.5, 1], fontsize=fontsize)
plt.xlabel('Frequency (GHz)', fontsize=fontsize, labelpad=2)
plt.ylabel('Integrated histogram', fontsize=fontsize)
ax.set_title('Qubit '+r'$f_{10}$', fontsize=fontsize, pad=4)
ax.text(4.1,
        0.47,
        'Median:',
        fontsize=fontsize,
        color='k',
        horizontalalignment='left',
        verticalalignment='center')
ax.text(4.1,
        0.35,
        '%.2f GHz'%np.round(np.median(f10_array), 2),
        fontsize=fontsize,
        color='C0',
        horizontalalignment='left',
        verticalalignment='center')

# T1 ecdf
ax_idx = 1
ax = fig.add_axes([ax_xs[ax_idx], ax_ys[ax_idx], ax_width[ax_idx], ax_height[ax_idx]])
T1_array = np.sort(np.array(sq_property['T1 (us)']))
y = np.arange(len(T1_array))/len(T1_array)
ax.step(np.hstack([T1_array, T1_array[-1]]), np.hstack([y, 1]), color='C0')
ax.plot([np.median(T1_array)]*100, np.linspace(0, 1, 100), color='C0', ls='--')
plt.xticks([0, 100, 200],fontsize=fontsize-1)
plt.yticks([0, 0.5, 1], [])
ax.set_xlim(0, 210)
ax.set_ylim(0, 1)
plt.xlabel(r'$T_1$'+' ('+r'$\mu$'+'s)', fontsize=fontsize, labelpad=2)
ax.set_title('Qubit '+r'$T_1$', fontsize=fontsize, pad=4)
ax.text(120,
        0.47,
        'Median:',
        fontsize=fontsize,
        color='k',
        horizontalalignment='left',
        verticalalignment='center')
ax.text(120,
        0.35,
        '%.1f '%np.round(np.median(T1_array), 1)+r'$\mu$'+'s',
        fontsize=fontsize,
        color='C0',
        horizontalalignment='left',
        verticalalignment='center')

# T2 ecdf
ax_idx = 2
ax = fig.add_axes([ax_xs[ax_idx], ax_ys[ax_idx], ax_width[ax_idx], ax_height[ax_idx]])
T2_array = np.sort(np.array(sq_property['T2 (us)']))
y = np.arange(len(T2_array))/len(T2_array)
ax.step(np.hstack([T2_array, T2_array[-1]]), np.hstack([y, 1]), color='C0')
ax.plot([np.median(T2_array)]*100, np.linspace(0, 1, 100), color='C0', ls='--')
plt.xticks(fontsize=fontsize-1)
plt.yticks([0, 0.5, 1], [])
ax.set_xlim(0, 50)
ax.set_ylim(0, 1)
plt.xlabel(r'$T_2^{SE}$'+' ('+r'$\mu$'+'s)', fontsize=fontsize, labelpad=2)
ax.set_title('Qubit '+r'$T_2^{SE}$', fontsize=fontsize, pad=4)
ax.text(27,
        0.47,
        'Median:',
        fontsize=fontsize,
        color='k',
        horizontalalignment='left',
        verticalalignment='center')
ax.text(27,
        0.35,
        '%.1f '%np.round(np.median(T2_array), 1)+r'$\mu$'+'s',
        fontsize=fontsize,
        color='C0',
        horizontalalignment='left',
        verticalalignment='center')

# operation ecdf
ax_idx = 3
ax = fig.add_axes([ax_xs[ax_idx], ax_ys[ax_idx], ax_width[ax_idx], ax_height[ax_idx]])
# sq gate
sq_error_array = np.sort(np.array(sq_property['Single-qubit gate error']))
y = np.arange(len(sq_error_array))/len(sq_error_array)
ax.step(np.hstack([sq_error_array, sq_error_array[-1]]), np.hstack([y, 1]), color='C0')
ax.plot([np.median(sq_error_array)]*100, np.linspace(0, 1, 100), color='C0', ls='--')

# tq gate
CZ_error_array = np.hstack([np.array(cz_error_diverging_flow).flatten(), np.array(cz_error_vortex).flatten()])
CZ_error_array = CZ_error_array[np.logical_not(np.isnan(CZ_error_array))]
CZ_error_array = np.sort(CZ_error_array)
y = np.arange(len(CZ_error_array))/len(CZ_error_array)
ax.step(np.hstack([CZ_error_array, CZ_error_array[-1]]), np.hstack([y, 1]), color='C1')
ax.plot([np.median(CZ_error_array)]*100, np.linspace(0, 1, 100), color='C1', ls='--')

# readout
readout_error_array = np.sort(np.array(1-(sq_property['|0> readout fidelity']+sq_property['|1> readout fidelity'])/2))
y = np.arange(len(readout_error_array))/len(readout_error_array)
ax.step(np.hstack([readout_error_array, readout_error_array[-1]]), np.hstack([y, 1]), color='C2')
ax.plot([np.median(readout_error_array)]*100, np.linspace(0, 1, 100), color='C2', ls='--')

ax.set_xlim(0.00005, 0.02)
ax.set_ylim(0, 1)
plt.xscale('log')
plt.xticks(fontsize=fontsize-1)
plt.yticks([0, 0.5, 1], [])
plt.xlabel('Error', fontsize=fontsize, labelpad=2)
ax.set_title('Operation error', fontsize=fontsize, pad=4)
ax.text(0.000065,
        0.77,
        'Median:',
        fontsize=fontsize,
        color='k',
        horizontalalignment='left',
        verticalalignment='center')
ax.text(0.000065,
        0.7,
        'SQ:\n'+r'%.1f$\times 10^{-4}$'%np.round(np.median(sq_error_array)*10000, 2),
        fontsize=fontsize,
        color='C0',
        horizontalalignment='left',
        verticalalignment='top')
ax.text(0.000065,
        0.5,
        'CZ:\n'+r'%.1f$\times 10^{-3}$'%np.round(np.median(CZ_error_array)*1000, 2),
        fontsize=fontsize,
        color='C1',
        horizontalalignment='left',
        verticalalignment='top')
ax.text(0.000065,
        0.3,
        'Readout:\n'+r'%.1f$\times 10^{-3}$'%np.round(np.median(readout_error_array)*1000, 2),
        fontsize=fontsize,
        color='C2',
        horizontalalignment='left',
        verticalalignment='top')


file_name = "SuppFigS4_mpl.pdf"
plt.savefig(file_name,
            dpi=None,
            facecolor='None',
            edgecolor='None',
            orientation='portrait',
            format="pdf",
            bbox_inches='tight')

In [7]:
print('sq:', 1-np.median(sq_error_array)/1.5)
print('tq:', 1-np.median(CZ_error_array)/1.25)
print('readout:', 1-np.median(readout_error_array))

sq: 0.9997212274277039
tq: 0.9966588120271693
readout: 0.9933937800056203
